# Graph theory to model Ticket to ride

<a target="_blank" href="https://colab.research.google.com/github/mgfernan/mgfernan.github.io/blob/master/docs/assets/Ticket_to_ride_graph_analysis.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook contains the analysis of the board game [Ticket to Ride](https://boardgamegeek.com/boardgame/9209/ticket-ride) using graph theory. To model the game, a distribution of various elements will be required (number of tracks, distribution of colors, and distribution of track lengths per each color).

This notebook makes use of the following key libraries:

- [Pandas](https://pandas.pydata.org/) for data loading and processing
- [NetworkX](https://networkx.org/) for graph processing and above all, to **find the shortest path between two stations** (nodes).

First we will need to install some required libraries as well as import necessary modules


In [ ]:
!pip install matplotlib networkx pandas

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd

The following cells contain some constants used in the notebook as well as the links to the CSVs that contain the Tracks and routes

In [ ]:
TRACK_LENGTH_COLUMN = 'length'
FROM_COLUMN = 'from'
TO_COLUMN = 'to'

TRACKS_CSV_FILE_URL = 'https://raw.githubusercontent.com/mgfernan/mgfernan.github.io/master/docs/assets/Ticket_to_ride%20-%20Tracks.csv'
TICKETS_CSV_FILE_URL = 'https://raw.githubusercontent.com/mgfernan/mgfernan.github.io/master/docs/assets/Ticket_to_ride%20-%20Tickets.csv'

First let's load the Track CSV, this will be used to visualize the track network as a mathematical graph (of nodes/stations and edges/tracks). The CSV contains the two stations for each track as well as their associated color and lengths (number of wagons required to complete the track)

In [ ]:
df = pd.read_csv(TRACKS_CSV_FILE_URL)
df[TRACK_LENGTH_COLUMN] = pd.to_numeric(df[TRACK_LENGTH_COLUMN])
df.head()

With this `DataFrame` we can know useful information such as:

- Number of total tracks
- Number of wagons required for each color
- Track distribution for each color

In addition, we can use the `DataFrame` to create the **graph** using the `networkx` library and make a visual representation of the track network

In [ ]:
# Create a directed graph with weighted edges
graph = nx.from_pandas_edgelist(df, FROM_COLUMN, TO_COLUMN, edge_attr=TRACK_LENGTH_COLUMN, create_using=nx.Graph())

# Plot the graph
pos = nx.spring_layout(graph, weight=TRACK_LENGTH_COLUMN, seed=4)
nx.draw(graph, pos, with_labels=True, node_size=400, node_color='skyblue', font_size=9, font_color='black', font_weight='bold', edge_color='grey', linewidths=2)

# Add edge labels showing weights
edge_labels = nx.get_edge_attributes(graph, TRACK_LENGTH_COLUMN)
nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels)

# Display the plot
plt.show()

The advantage of modeling the track network as a graph is that we can use already existing libraries to compute the shortest path between two stations (both the passing stations as well as the total length), as in the following example

In [ ]:
FROM = 'El Paso'
DESTINATION = 'Raleigh'

shortest_path = nx.shortest_path(graph, source=FROM, target=DESTINATION, weight=TRACK_LENGTH_COLUMN)
print(f'The shortest path goes through these stations: {" - ".join(shortest_path)}' )

shortest_path_length = nx.shortest_path_length(graph, source=FROM, target=DESTINATION, weight=TRACK_LENGTH_COLUMN)
print(f'The length for the shortest path is: {shortest_path_length}')


These methods (specially `nx.shortest_path_length`) are very relevant to reverse engineer the ticket cards (that give points if a player completes a route). Basically we need to know how the points awarded per card is related to the shortest path.

To do so, we will load the Ticket CSV into a `DataFrame` and compute the shortest path for each card:

In [ ]:
# Load the Ticket CSV
df_tickets = pd.read_csv(TICKETS_CSV_FILE_URL)

# Create a new column in the dataframe with the shortest path length for each route (i.e. ticket card)
df_tickets[TRACK_LENGTH_COLUMN] = df_tickets.apply(lambda r: nx.shortest_path_length(graph, source=r[FROM_COLUMN], target=r[TO_COLUMN], weight=TRACK_LENGTH_COLUMN), axis=1)

df_tickets.head()

At this point we can realize that the points awarded per route are in fact equivalent (at least on the most cases) with the lengths (i.e. total number of wagons required to complete the route), which makes sense.

In [ ]:
plt.plot(df_tickets[TRACK_LENGTH_COLUMN], df_tickets['points'], 'o')
plt.xlabel('Shortest route length [number of wagons]')
plt.ylabel('Ticket card points')
plt.title('Relationship between Ticket card points and\nShortest route length')
plt.show()